In [1]:
#Student scraper > XML output

from urllib.request import urlopen
import certifi
import ssl
import json
import sys
from urllib.error import HTTPError, URLError
import time

BLANK_PAGE_SZ = 6350   # Not Found Web page size
magazine = "Student"
urlpath  = "https://pretraziva.rs/pregled/student"
urlpath2 = "https://pretraziva.rs/prikaz/student"
urlpath3 = "https://pretraziva.rs/download/student--"

# Make a list of all available issues

# Get a list of all issues from the JavaScript variable "publicationYears"
str1 = urlopen(urlpath, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
htmlstr = str1.read().decode('utf-8')
i1 = htmlstr.index("{")
i2 = htmlstr.index("};")
dictionary = eval(htmlstr[i1:i2+1])

# Print the information

print("\nYears:", flush=True)
for y in dictionary:
    print(y, end=" ", flush=True)

print("\nAll issues by date:", flush=True)
for y in dictionary:
    print(y)
    issues = dictionary[y]
    for i in issues:
        print("  " + i[1].replace(" ","") + y + ".", flush=True)
print()

# Move the text of an issue to a file

# Iterate all pages by year/issue and write the plain text to new files
for y in dictionary:    # years
    
    issues = dictionary[y]

    for i in issues:    # issues
        ymd = y + "-" + i[1][4:6] + "-" + i[1][0:2]
        iurlpath = urlpath2 + "/" + ymd
        print(y, iurlpath, flush=True)
        
        # Iterate pages

        # Number of pages
        iurlpath2 = iurlpath + "/1"
        istr2 = urlopen(iurlpath2, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
        html2 = istr2.read().decode('utf-8')
        i1 = html2.index("var lastPage = ")
        lp = html2[i1+15:i1+19]
        i2 = lp.index(";")
        npg = eval(lp[:i2])
        print(npg, "pages", flush=True)
        
        # Write text to a new file
        ioutpath = ymd + " " + magazine + ".xml"
        print("Writing text to:", ioutpath, flush=True)
        f = open(ioutpath, "w", encoding='utf-8',newline='')
        
        # Add TEI header
        print('Adding header', flush=True)
        with open("TEI header.txt") as header:
            for line in header:
                f.write(line)

        for pg in range(1, npg+1):
            print("  page", pg, flush=True)
            try:
                try:
                    iurlpath3 = urlpath3 + ymd + "--" + str(pg) + ".txt"
                    str3 = urlopen(iurlpath3, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
                    txt  = str3.read().decode('utf-8')
                    f.write(txt)
                except:
                    iurlpath3 = urlpath3 + ymd + "--0" + str(pg) + ".txt"
                    str3 = urlopen(iurlpath3, timeout=10, context=ssl.create_default_context(cafile=certifi.where()))
                    txt  = str3.read().decode('utf-8')
                    f.write(txt)
            except:
                pass
        
        # Add TEI closed tags at the end
        print('Adding footer', flush=True)
        with open("TEI closed tags.txt") as footer:
            for line in footer:
                f.write(line)
    
        f.close()
        
print('Done.', flush=True)


Years:
1937 1938 1939 1940 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1968 1992 1993 1994 1995 
All issues by date:
1937
  15.03.1937.
  05.04.1937.
  26.04.1937.
  27.05.1937.
  06.07.1937.
  27.09.1937.
  15.10.1937.
  06.11.1937.
  11.12.1937.
1938
  01.02.1938.
  01.03.1938.
  15.03.1938.
  08.04.1938.
  25.06.1938.
1939
  01.02.1939.
  06.05.1939.
  01.06.1939.
  20.08.1939.
  24.10.1939.
  07.12.1939.
  23.12.1939.
1940
  01.02.1940.
  04.06.1940.
  15.07.1940.
  06.11.1940.
1946
  26.02.1946.
  23.03.1946.
  12.04.1946.
  29.04.1946.
  17.05.1946.
  06.06.1946.
  20.06.1946.
  26.07.1946.
  11.10.1946.
  18.10.1946.
  25.10.1946.
  01.11.1946.
  08.11.1946.
  15.11.1946.
  22.11.1946.
  27.11.1946.
  06.12.1946.
  13.12.1946.
  21.12.1946.
  27.12.1946.
1947
  10.02.1947.
  15.02.1947.
  21.02.1947.
  28.02.1947.
  09.03.1947.
  15.03.1947.
  22.03.1947.
  28.03.1947.
  04.04.1947.
  16.04.1947.
  23.04.1947.
  05.05.1947.
  14.05.1947.
  23.05.1947.
  30.05.1947.
 

In [2]:
#Scuffed word count 

import os, glob

total = 0
filecount = 0

for filename in glob.glob('*.xml'):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        filecount = filecount + 1
        data = f.read()
        words = data.split()
        total = total + len(words)
        
total = total - filecount*57       # 57 is the WC in the two TEI files
print(total)

6442751
